许多读者在实战中发现，静态中断（interrupt_before / interrupt_after）虽然可以作为“断点”来观察状态，但它并不能实现我们真正需要的人机交互（Human-in-the-Loop, HITL）功能，例如：
- 在 Agent 执行关键操作（如删除数据库、调用付费 API）前，人工“批准”或“拒绝”。
- 在 Agent 生成初稿后，人工“审查”并“编辑” 其内容。
- 在 Agent 需要额外信息时，人工“输入” 所需的数据。

这些功能是构建可靠、可控 Agent 的基石。值得注意的是，LangGraph 提供了另一套更强大的机制来解决这个问题，这就是动态中断 (Dynamic Interrupts)。

本期教程将作为一篇补充内容，专门深入解析 LangGraph 的人机交互机制，帮你彻底分清两种“中断”的区别，并掌握如何构建真正需要人工干预的“审批”流程。 在本期教程中，你将掌握：
- 两种中断模式的对比：静态中断（调试）vs 动态中断（人机交互）。
- 动态中断组件Checkpointer, thread_id, interrupt(), Command。
- 综合实战：从零构建一个需要人工批准的“审批”工作流。
- 黄金法则：使用动态中断时必须遵守的“幂等性”与“节点重跑”原则。

# 第一部分: 重新定义“中断” - 两种模式的对比
要掌握人机交互，我们必须首先厘清 LangGraph 中两种“中断”的根本区别。它们的设计目的、使用方法和恢复机制截然不同。
## 模式一：静态中断 (Static Interrupts) - “调试断点”
定义方式: 在 graph.compile() 或 graph.invoke() 时，作为参数传入。

```
# 在编译时设置
graph = builder.compile(
    interrupt_before=["node_name"]
)
# 或在运行时设置
graph.invoke(..., interrupt_after=["node_name"])
```

核心目的: 调试 (Debugging)。它允许开发者在某个节点执行前或执行后暂停图，以便检查（get_state）当时的 State 状态，类似于 IDE 中的“断点”。

恢复方式:

```
# 传入 None 来恢复，表示“继续执行”
graph.invoke(None, config=config)
```

局限性: 这是一个“只出不进”的暂停。它无法在恢复时将“人工决策”（如 True 或 "已修改的内容") 传递回 节点内部。因此，它不能用于真正的人机交互。

## 模式二：动态中断 (Dynamic Interrupts) - “人机交互” (HITL)

定义方式: 在节点函数内部，直接调用 langgraph.types.interrupt 函数。


```
from langgraph.types import interrupt

def approval_node(state: AgentState):
# ...
# 在代码逻辑中动态触发
    decision = interrupt("请批准此操作")
# ...
```

核心目的: 人机交互 (Human-in-the-Loop)。它允许图在运行时根据特定逻辑暂停，等待用户的输入（如批准、编辑、提供数据）。 恢复方式:

```
from langgraph.types import Command

# 传入 Command(resume=...) 来恢复
# resume 传入的值将成为 interrupt() 的返回值
graph.invoke(Command(resume=True), config=config)
```

强大之处: 这是一个“有进有出”的暂停。它不仅暂停，还能在恢复时接收一个值（resume=... 传入的值），这个值会成为 interrupt() 函数的返回值（上例中的 decision 变量），从而驱动后续的业务逻辑。

# 第二部分: “动态中断”的四个核心组件
要使“动态中断”按预期工作，我们必须同时使用到四个核心组件，它们缺一不可。
## Checkpointer (状态记录)
- 职责：持久化。当 interrupt() 被调用时，Checkpointer 负责将当前 Graph 的完整状态保存到数据库（如 MemorySaver, SqliteSaver, RedisSaver）。
- 关键：没有 Checkpointer，动态中断无法工作。Graph 必须有办法“存档”，才能在未来“读档”并恢复。
## Config 中的 thread_id (会话 ID)
- 职责：唯一标识。thread_id 就像是你的“游戏存档文件名”。
- 关键：你必须使用一个固定的 thread_id (通过 config={"configurable": {"thread_id": "..."}} 传入) 来调用 Graph。当恢复时，LangGraph 才知道要加载哪一个被暂停的会话。
## interrupt() (暂停函数)
- 职责：执行暂停。在节点中调用它时，它会：
1. 抛出一个特殊信号，通知 LangGraph 框架“暂停”。
2. 框架命令 Checkpointer 保存当前 thread_id 的状态。
3. invoke() 调用立即返回。interrupt() 中传递的参数（如 "请批准"）会包含在返回结果的 __interrupt__ 字段中，用于展示给用户。
## Command(resume=…) (恢复指令)
- 职责：恢复执行。当用户做出决策后，再次调用 graph.invoke()，但这次传入的不是输入数据，而是一个 Command(resume=...) 对象。
- 关键：传入 resume 的值（如 True, False 或一个包含编辑后文本的字典）将在节点恢复执行时，被 interrupt()函数捕获并作为其返回值。

In [4]:
!pip install langgraph langchain

  Using cached langgraph-1.0.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached langgraph_checkpoint-3.0.1-py3-none-any.whl.metadata (4.7 kB)
  Using cached langgraph_prebuilt-1.0.2-py3-none-any.whl.metadata (5.0 kB)
  Using cached langgraph_sdk-0.2.9-py3-none-any.whl.metadata (1.5 kB)
  Using cached ormsgpack-1.12.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
INFO: pip is looking at multiple versions of langgraph-prebuilt to determine which version is compatible with other requirements. This could take a while.
  Using cached langgraph_checkpoint-3.0.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached langgraph_checkpoint-2.1.2-py3-none-any.whl.metadata (4.2 kB)
  Using cached langgraph_checkpoint-2.1.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached langgraph_checkpoint-2.1.0-py3-none-any.whl.metadata (4.2 kB)
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
INFO: pip is still looking at multiple versions of langgra

# 第三部分: 综合实战 - 构建“人工审批”流程
现在，我们从零开始构建一个“转账审批”流程，演示这四个组件如何协同工作。
## 3.1 基础设置与状态定义我们定义
一个简单的 State，包含操作详情和当前状态。



In [5]:
import time
from typing import TypedDict, Literal, Optional
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import Command, interrupt

class AgentState(TypedDict):
    action_details: str
    status: Optional[Literal["pending", "approved", "rejected"]]

## 3.2 节点定义 (审批节点)
这是实战的核心。我们将 interrupt() 嵌入到节点逻辑中。

In [7]:
def approval_node(state: AgentState):
    """
    一个需要人工批准的节点。

    注意：这个节点在“恢复”时会从头重新运行。
    """
    print("--- 节点 [approval_node] 开始执行 ---")
    print(f"  > 待办操作: {state['action_details']}")

    # 仅在 'pending' 状态时才触发中断
    if state['status'] == 'pending':
        # 调用 interrupt() 来暂停
        # 'payload' 将会返回给调用者
        payload = {
            "question": "您是否批准此操作？",
            "details": state["action_details"]
        }

    print("  > 暂停，等待人工批准...")
    # 第一次运行：Graph 在此暂停。
    # 恢复运行时：decision 将被赋予 Command(resume=...) 中的值。
    decision = interrupt(payload)

    print(f"  > 收到人工决策: {decision}")

    # 根据决策更新状态
    if decision:
        return {"status": "approved"}
    else:
        return {"status": "rejected"}

    # 如果状态不是 'pending' (例如在重跑时)，则跳过
    print(f"  > 状态为 {state['status']}, 跳过中断。")
    return {}

def proceed_node(state: AgentState):
    print("--- 节点 [proceed_node] 执行 ---")
    print(f"正在执行操作: {state['action_details']}")
    return {}

def cancel_node(state: AgentState):
    print("--- 节点 [cancel_node] 执行 ---")
    print(f"取消操作: {state['action_details']}")
    return {}

## 3.3 构建 Graph (带条件路由)
我们使用条件路由，根据 approval_node 之后的 status 状态决定下一步。

In [8]:
builder = StateGraph(AgentState)

# 添加节点
builder.add_node("approval", approval_node)
builder.add_node("proceed", proceed_node)
builder.add_node("cancel", cancel_node)

# 设置入口
builder.set_entry_point("approval")

# 定义条件路由
def route_decision(state: AgentState):
    if state["status"] == "approved":
        return"proceed"
    else:
        return"cancel"

# 'approval' 节点完成后，根据 'status' 决定去向
builder.add_conditional_edges(
    "approval",
    route_decision,
    {
        "proceed": "proceed",
        "cancel": "cancel"
    }
)

# 最终节点
builder.add_edge("proceed", END)
builder.add_edge("cancel", END)

## 3.4 编译与执行（暂停与恢复）
这是演示“暂停-恢复”流程的关键。

In [9]:
# 1. 核心组件：实例化 Checkpointer
checkpointer = MemorySaver()

# 编译 Graph，必须传入 checkpointer
graph = builder.compile(checkpointer=checkpointer)

# 2. 核心组件：定义一个唯一的 thread_id
config = {"configurable": {"thread_id": "tx-12345"}}
initial_input = {"action_details": "向用户 'A' 转账 500RMB", "status": "pending"}

# 第一次调用，触发暂停
print("--- 第一次运行 (将触发暂停) ---")
# Graph 会运行到 approval_node，调用 interrupt()，然后暂停
result = graph.invoke(initial_input, config=config)

print("\n--- Graph 已暂停 ---")
print("  > Graph 的当前状态 (已保存):")
print(f"  > {graph.get_state(config).values}")
print("\n  > 'interrupt' 返回的数据 (用于展示给用户):")
# 注意这个特殊的 `__interrupt__` 字段
print(f"  > {result['__interrupt__']}")

# 此时应用程序（如 Web UI）会向用户展示 'result['__interrupt__']' 的内容
# 用户审查后，决定 "批准" (True)
time.sleep(1)
human_decision = True
print(f"\n--- 用户已做出决策: {human_decision} ---")

# 第二次调用，使用 Command 恢复
print("--- 恢复 Graph 运行 ---")

# 使用 Command(resume=...) 和 *相同的 config* 来恢复
# 传入的 `resume=True` 将成为 `interrupt()` 的返回值
resume_result=graph.invoke(Command(resume=human_decision), config=config)

print("\n--- Graph 运行完毕 ---")
print("  > Graph 的最终状态:")
print(f"  > {graph.get_state(config).values}")
print("\n  > 最后一步的输出:")
print(f"  > {resume_result}")

--- 第一次运行 (将触发暂停) ---
--- 节点 [approval_node] 开始执行 ---
  > 待办操作: 向用户 'A' 转账 500RMB
  > 暂停，等待人工批准...

--- Graph 已暂停 ---
  > Graph 的当前状态 (已保存):
  > {'action_details': "向用户 'A' 转账 500RMB", 'status': 'pending'}

  > 'interrupt' 返回的数据 (用于展示给用户):
  > [Interrupt(value={'question': '您是否批准此操作？', 'details': "向用户 'A' 转账 500RMB"}, id='17b3d2fb0abea3a10b99d9dc33454ff7')]

--- 用户已做出决策: True ---
--- 恢复 Graph 运行 ---
--- 节点 [approval_node] 开始执行 ---
  > 待办操作: 向用户 'A' 转账 500RMB
  > 暂停，等待人工批准...
  > 收到人工决策: True
--- 节点 [proceed_node] 执行 ---
正在执行操作: 向用户 'A' 转账 500RMB

--- Graph 运行完毕 ---
  > Graph 的最终状态:
  > {'action_details': "向用户 'A' 转账 500RMB", 'status': 'approved'}

  > 最后一步的输出:
  > {'action_details': "向用户 'A' 转账 500RMB", 'status': 'approved'}


# 第四部分: 黄金法则 - 动态中断的“天坑”
动态中断非常强大，但也引入了一个最容易出错的“天坑”：节点重跑。 当调用 Command(resume=...) 恢复时，LangGraph 不会从 interrupt() 函数的那一行代码继续执行，而是会从头开始重新执行包含 interrupt() 的整个节点函数（即 approval_node）。 这带来了两个必须遵守的“黄金法则”：
## 幂等性 (Idempotency)
规则：绝对不能在 interrupt() 调用之前放置任何“有副作用”且“非幂等”的操作（如写入数据库、发送 API 请求）。

举个反例 (错误):


```
def bad_node(state: AgentState):
    # 错误！这个操作会执行两次！
    # 第一次是暂停前，第二次是恢复后
    db.append_to_log("Approval process started...")

    decision = interrupt("Approve this action?")

    if decision:
        db.execute_transfer(...)
    return ...
```



在上面的例子中，db.append_to_log 会在暂停前运行一次，在恢复后（节点重跑时）再次运行，导致数据库中出现重复日志。

正确做法:

- 做法 A（推荐）：将所有“副作用”操作放在 interrupt() 之后，并由其返回值控制。
- 做法 B：将“副作用”拆分到单独的节点中（如 proceed_node），利用路由来确保它只在批准后执行一次。
## 状态驱动 (State-Driven Logic)
规则：由于节点会重跑，你必须使用 State 来防止 interrupt() 被重复触发。 在我们的实战代码中，我们正是这么做的：

```
def approval_node(state: AgentState):
    print("--- 节点 [approval_node] 开始执行 ---")

    # 正确：使用状态来控制中断
    if state['status'] == 'pending':
        # 第一次运行：'pending'，触发中断
        decision = interrupt(...)
        if decision:
            return {"status": "approved"}
        else:
            return {"status": "rejected"}

    # 第二次运行 (恢复后)：
    print(f"  > 状态为 {state['status']}, 跳过中断。")
    return {}
```



这种模式确保了 interrupt() 在整个会话中只被触发一次。
# 本期总结
在本期补充教程中，我们彻底理清了 LangGraph 的两种中断机制：
- 静态中断 (interrupt_before): 仅用于调试的“断点”，无法实现人机交互。
- 动态中断 (interrupt()): 专为人机交互 (HITL) 设计，是构建审批、编辑、验证流程的核心。

我们通过一个“审批”实战，掌握了实现动态中断的四个组件Checkpointer, thread_id, interrupt, Command，并深刻理解了“节点重跑”和“幂等性”这两条黄金法则。 掌握了动态中断，你的 Agent 才真正从一个“自动化脚本”进化为了一个可控、可靠的“智能助手”。